In [1]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

# Library imported

In [2]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim

from nltk.corpus import stopwords 
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from torch.nn.utils.rnn import pad_sequence
import string

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder

is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

2025-11-26 09:36:16.043313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764149776.263191      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764149776.327998      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

GPU is available


# SentenceTransformer (all-MiniLM-L6-v2) from Huggingface

In [3]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")   # 384-dim embeddings
EMB_SIZE = embedder.get_sentence_embedding_dimension()
print("Embedding size:", EMB_SIZE)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding size: 384


In [4]:
df = pd.read_csv("/kaggle/input/mobile-reviews-sentiment-and-specification/Mobile Reviews Sentiment.csv")

In [5]:
df.shape

(50000, 25)

# Clean Text Preprocessing Function

In [6]:
def clean_text(text):
    # remove html
    text = re.sub(r"<.*?>", "", text)
    # remove url
    text = re.sub(r"http\S+", "", text)
    # remove punctuation + emoji
    text = re.sub(r"[^A-Za-z0-9\s]", "", text)
    # remove digits
    text = re.sub(r"\d+", "", text)
    # normalize spaces
    text = re.sub(r"\s+", " ", text)
    # lower case
    text = text.lower().strip()
    return text


# Dataset : returns sequence of embeddings

We use SBERT’s tokenizer internally to get subword tokens, then embed each token separately → resulting in a sequence.

In [7]:
class HybridEmbeddingDataset(Dataset):
    def __init__(self, texts, labels, embedder):
        self.texts = [clean_text(t) for t in texts]
        self.labels = labels
        self.embedder = embedder

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]

        # Get token-level embeddings (not pooled)
        token_embeddings = self.embedder.encode(
            text,
            output_value="token_embeddings",      # <-- IMPORTANT
            convert_to_tensor=True
        )  # shape: (seq_len, 384)

        label = torch.tensor(self.labels[idx], dtype=torch.long)

        return token_embeddings, label


# Collate function for padding

In [8]:
def collate_batch(batch):
    sequences = [item[0] for item in batch]
    labels = torch.tensor([item[1] for item in batch], dtype=torch.long)

    # Pad sequences to same length
    padded = nn.utils.rnn.pad_sequence(sequences, batch_first=True)

    return padded, labels


In [10]:
texts = df["review_text"].astype(str)


In [11]:
texts

0           Not worth the money spent. Wouldn’t recommend.
1        Absolutely love this phone! The camera is next...
2        Loving the clean UI and fast updates. Loving i...
3        Build quality feels solid and durable. No regr...
4        Not bad for daily use but could be optimized. ...
                               ...                        
49995    Battery easily lasts a day with heavy use. Lov...
49996    Battery life could be slightly better. Fine bu...
49997    Face unlock is instant, super smooth. Best pur...
49998    Build quality feels solid and durable. Loving ...
49999    Loving the clean UI and fast updates. Loving i...
Name: review_text, Length: 50000, dtype: object

# train test split 

In [13]:
y = df["sentiment"]
y

0        Negative
1        Positive
2        Positive
3        Positive
4         Neutral
           ...   
49995    Positive
49996     Neutral
49997    Positive
49998    Positive
49999    Positive
Name: sentiment, Length: 50000, dtype: object

In [18]:
le = LabelEncoder()
y_enc = le.fit_transform(y)
print(y_enc[:10])

[0 2 2 2 1 2 2 0 2 2]


In [21]:
x_train,x_test,y_train,y_test = train_test_split(texts,y_enc,test_size=0.15, random_state=12)

x_train.shape

(42500,)

In [22]:

train_dataset = HybridEmbeddingDataset(x_train, y_train, embedder)
test_dataset  = HybridEmbeddingDataset(x_test,  y_test,  embedder)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, collate_fn=collate_batch)
test_loader  = DataLoader(test_dataset,  batch_size=512, shuffle=False, collate_fn=collate_batch)


# Hybrid Model (BiLSTM + Attention)

In [23]:
class BiLSTM_Attention(nn.Module):
    def __init__(self, embed_dim=384, hidden_dim=256, num_classes=3):
        super().__init__()
        
        self.lstm = nn.LSTM(
            embed_dim, 
            hidden_dim, 
            batch_first=True,
            bidirectional=True
        )

        # Attention layer
        self.attention = nn.Linear(hidden_dim * 2, 1)

        # Final classifier
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)   # (B, L, 512)

        # Attention weights
        attn_weights = F.softmax(self.attention(lstm_out), dim=1)  # (B, L, 1)

        # Context vector: sum(LSTM_out * attention_weights)
        context = torch.sum(attn_weights * lstm_out, dim=1)  # (B, 512)

        return self.fc(context)


# LSTM model taining

In [24]:
model = BiLSTM_Attention().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [25]:
def train_model(model, train_loader, val_loader, epochs=5):
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in tqdm(train_loader):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            pred = outputs.argmax(1)
            correct += (pred == y_batch).sum().item()
            total += y_batch.size(0)

        train_acc = correct / total

        # ---- Validation ----
        model.eval()
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for X_batch, y_batch in tqdm(val_loader):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                pred = outputs.argmax(1)

                val_correct += (pred == y_batch).sum().item()
                val_total += y_batch.size(0)

        val_acc = val_correct / val_total
        model.train()

        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss:.3f} | "
              f"Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f}")


In [26]:
train_model(model, train_loader, test_loader, epochs=5)

100%|██████████| 15/15 [00:46<00:00,  3.08s/it]


Epoch 1/10 | Loss: 46.685 | Train Acc: 0.767 | Val Acc: 1.000


100%|██████████| 15/15 [00:45<00:00,  3.02s/it]


Epoch 2/10 | Loss: 0.274 | Train Acc: 1.000 | Val Acc: 1.000


100%|██████████| 15/15 [00:44<00:00,  2.95s/it]


...

Epoch 8/10 | Loss: 0.005 | Train Acc: 1.000 | Val Acc: 1.000


100%|██████████| 15/15 [00:46<00:00,  3.10s/it]


Epoch 9/10 | Loss: 0.004 | Train Acc: 1.000 | Val Acc: 1.000


100%|██████████| 15/15 [00:45<00:00,  3.04s/it]

Epoch 10/10 | Loss: 0.003 | Train Acc: 1.000 | Val Acc: 1.000


# Inference

In [31]:
def predict_sentiment(text):
    model.eval()

    text = clean_text(text)

    token_emb = embedder.encode(
        text, 
        output_value="token_embeddings",
        convert_to_tensor=True
    ).unsqueeze(0).to(device)   # (1, L, 384)

    logits = model(token_emb)
    pred = torch.argmax(logits, dim=1).item()

    mapping = ["Negative","Neutral","Positive"]  #  numeric to string
    return mapping[pred]


In [33]:
predict_sentiment(" phone service very bad")

'Negative'

# Save the Model (Recommended Way)

In [38]:
def save_model(model, path="sentiment_model.pth", label2id=None, config=None):
    save_dict = {
        "model_state_dict": model.state_dict(),
        # "label2id": label2id,
        # "config": config
    }
    torch.save(save_dict, path)
    print(f"Model saved to {path}")


In [39]:
# config = {
#     "embedding_dim": 384,
#     "hidden_dim": 256,
#     "num_classes": 3,
# }

# label2id = {"negative": 0,  "neutral": 1, "positive": 2}

save_model(model, "hybrid_sentiment.pth")


Model saved to hybrid_sentiment.pth


In [40]:
def load_model(model_class, path="sentiment_model.pth"):
    checkpoint = torch.load(path, map_location="cpu")
    
    # Rebuild model with stored config
   # model = model_class(**checkpoint["config"])
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()

    # label2id = checkpoint["label2id"]
    # id2label = {v: k for k, v in label2id.items()}

    return model


In [41]:
model = load_model(BiLSTM_Attention, "hybrid_sentiment.pth")

In [47]:
from sentence_transformers import SentenceTransformer
import torch

encoder = SentenceTransformer("all-MiniLM-L6-v2")

mapping = ["Negative","Neutral","Positive"] 


In [52]:
def predict_text(model, text):
    text = clean_text(text)

    token_emb = encoder.encode(
        text, 
        output_value="token_embeddings",
        convert_to_tensor=True
    ).unsqueeze(0).to(device)   # (1, L, 384)

    logits = model(token_emb)
    pred = torch.argmax(logits, dim=1).item()
    return mapping[pred]


print(predict_text(model, "phone service given very good"))

Negative


# Export to ONNX

In [59]:
def export_onnx(model, filename):
    model_cpu = model.to("cpu")      # FIX 1: move model to CPU
    model_cpu.eval()

    dummy_input = torch.randn(1, 1, 384)  # SentenceTransformer output size
                                          # already on CPU

    torch.onnx.export(
        model_cpu,
        dummy_input,
        filename,
        input_names=["embedding"],
        output_names=["logits"],
        dynamic_axes={"embedding": {0: "batch", 1: "seq_len"}},
        opset_version=17
    )

    print("Exported:", filename)


In [60]:
export_onnx(model, "hybrid_sentiment.onnx")

Exported: hybrid_sentiment.onnx


In [62]:
!pip install onnxruntime

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 80.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00


In [63]:
import onnxruntime as ort
import numpy as np
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-MiniLM-L6-v2")
ort_session = ort.InferenceSession("hybrid_sentiment.onnx")
id2label = {0: "negative", 2: "positive", 1: "neutral"}


In [65]:
def predict_onnx(text):
    # Step 1 → Encode text using SentenceTransformer
    emb = encoder.encode([text])              # shape (1, 768)
    emb = np.expand_dims(emb, axis=1)         # shape (1, 1, 768)

    # Step 2 → Run ONNX
    ort_inputs = {"embedding": emb.astype(np.float32)}
    logits = ort_session.run(None, ort_inputs)[0]

    # Step 3 → Get sentiment
    pred = np.argmax(logits, axis=1)[0]
    return id2label[pred]

print(predict_onnx("This product is very bad!"))


negative
